https://medium.freecodecamp.org/better-web-scraping-in-python-with-selenium-beautiful-soup-and-pandas-d6390592e251

In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

from bs4 import BeautifulSoup
import pandas as pd

import time
#import csv

In [3]:
reaction_testing_list = ['rxn03253', 'rxn02720', 'rxn00946', 'rxn02679', 'rxn02803']

# Step by Step

In [5]:
#Create an empty dictionary to fill with the reactions as keys and aliases as values
bigg_dict = {}
kegg_dict = {}

In [6]:
reaction = 'rxn03253'
url="http://modelseed.org/biochem/reactions/" + reaction
print(url)

http://modelseed.org/biochem/reactions/rxn03253


In [9]:
browser = webdriver.Firefox()
browser.get(url)

print(browser.current_url)

http://modelseed.org/biochem/reactions/rxn03253


In [10]:
soup = BeautifulSoup(browser.page_source, 'lxml')
print(soup)

<html class="ng-scope" lang="en" ng-app="ModelSEED"><head><style class="ng-binding" type="text/css">md-bottom-sheet.md-list md-list-item {    color: rgba(0,0,0,0.87);}md-bottom-sheet .md-subheader {    color: rgba(0,0,0,0.87);}md-card .md-card-image {    border-radius: 2px 2px 0 0;}md-checkbox .md-icon {  border-color: rgba(0,0,0,0.54);}md-checkbox:not([disabled]).md-primary .md-ripple {  color: rgb(0,172,193);}md-checkbox:not([disabled]).md-primary.md-checked .md-ripple {  color: rgb(117,117,117);}md-checkbox:not([disabled]).md-primary .md-icon {  border-color: rgba(0,0,0,0.54);}md-checkbox:not([disabled]).md-primary.md-checked .md-icon {  background-color: rgba(0,188,212,0.87);}md-checkbox:not([disabled]).md-primary.md-checked.md-focused .md-container:before {  background-color: rgba(0,188,212,0.26);}md-checkbox:not([disabled]).md-primary.md-checked .md-icon:after {  border-color: rgb(238,238,238);}md-checkbox[disabled] .md-icon {  border-color: rgba(0,0,0,0.26);}md-checkbox[disabled

In [11]:
table = soup.find_all('table')[0]

df = pd.read_html(str(table), header=None)  #header none--> first column = 0, second column = 1

browser.quit()

In [18]:
web_table = df[0]
#print(web_table)
aliases = web_table.loc[12][1]
aliases = aliases.split(';')
#print(aliases)
bigg_list = [i for i in aliases if 'BiGG:' in i]
kegg_list = [i for i in aliases if 'KEGG:' in i]

bigg_dict[reaction] = bigg_list
kegg_dict[reaction] = kegg_list
print(bigg_dict)
print(kegg_dict)

{'rxn03253': ['"BiGG:ACOAD4f"']}
{'rxn03253': ['"KEGG:R04754"']}


# Now as Function including testing of the function

In [20]:
def webrequest(reaction):
    if 'rxn' in reaction: #there are also other identifiers - that are not specific for SEED
        
        url="http://modelseed.org/biochem/reactions/" + reaction
        #print(url)

        browser = webdriver.Firefox()
        browser.get(url)

        #print(browser.current_url)

        try:
            element_present = EC.presence_of_element_located((By.TAG_NAME, 'table'))
            WebDriverWait(browser, 1000).until(element_present)
        except TimeoutException:
            print("1000s? Dein Ernst??")

        soup = BeautifulSoup(browser.page_source, 'lxml')
        #print(soup)

        table = soup.find_all('table')[0]

        df = pd.read_html(str(table), header=None)  #header none--> first column = 0, second column = 1


        web_table = df[0]
        aliases = web_table.loc[12][1]
        if type(aliases) == str:
            aliases = aliases.split(';')
            #print(aliases)
            bigg_list = [i for i in aliases if 'BiGG:' in i]
            kegg_list = [i for i in aliases if 'KEGG:' in i]
            #print(alias_list)
            bigg_dict[reaction] = bigg_list
            kegg_dict[reaction] = kegg_list

            with open('bigg_dict.txt', 'w') as f:
                for key, value in bigg_dict.items():
                    f.write('%s:\t%s\n' % (key, value))

            with open('kegg_dict.txt', 'w') as f:
                for key, value in kegg_dict.items():
                    f.write('%s:\t%s\n' % (key, value))

            browser.quit()


        else:
            browser.quit()


        #print(bigg_dict)
        #print(kegg_dict)

In [21]:
#Create an empty dictionary to fill with the reactions as keys and aliases as values
bigg_dict = {}
kegg_dict = {}

for react in reaction_testing_list:
    start = time.time()
    webrequest(react)
    end = time.time()
    print(end-start)

print(bigg_dict)
print(kegg_dict)

65.31271648406982
55.92931079864502
31.170374870300293
30.730515241622925
33.64790654182434
{'rxn03253': ['"BiGG:ACOAD4f"'], 'rxn02720': ['"BiGG:ACOAD5f"'], 'rxn00946': ['"BiGG:ACOAD7f"'], 'rxn02679': ['"BiGG:ACOAD3f"'], 'rxn02803': ['"BiGG:ACOAD6f"']}
{'rxn03253': ['"KEGG:R04754"'], 'rxn02720': ['"KEGG:R03857"'], 'rxn00946': ['"KEGG:R01279"'], 'rxn02679': ['"KEGG:R03777"'], 'rxn02803': ['"KEGG:R03990"']}


In [22]:
#Testing case for Reaction without aliases
webrequest('rxn00032') 

# And now import the model and check all reactions in it

In [24]:
import cobra

In [26]:
PA14_model = cobra.io.read_sbml_model('created_PA14.xml')
print(len(PA14_model.reactions))

1518


In [53]:
'''
All reactions are now requested and the extended dictionary is directly exported as txt file, 
since too many requests might be blocked and interrupted.
'''

#Create an empty dictionary to fill with the reactions as keys and aliases as values
bigg_dict = {}
kegg_dict = {}

for react in PA14_model.reactions:
    #start = time.time()
    webrequest(react.id)
    #end = time.time()
    #print(end-start)

#print(bigg_dict)
#print(kegg_dict)

WebDriverException: Message: Reached error page: about:neterror?e=dnsNotFound&u=http%3A//modelseed.org/biochem/reactions/rxn05465&c=UTF-8&f=regular&d=Die%20Verbindung%20mit%20dem%20Server%20modelseed.org%20schlug%20fehl.


In [58]:
index = 0
for i in range(len(PA14_model.reactions)):
    if PA14_model.reactions[i].id == 'rxn05465':
        print(PA14_model.reactions[i].name, index)
    index += 1

Malonyl-CoA-ACP transacylase 892


In [59]:
PA14_model.reactions[892].name

'Malonyl-CoA-ACP transacylase'

## Second half of the reactions

In [24]:
bigg_dict = {}
kegg_dict = {}

for i in range(892, len(PA14_model.reactions)):
    #start = time.time()
    webrequest(PA14_model.reactions[i].id)
    #end = time.time()
    #print(end-start)

# Combine the two dictionaries in one

### BiGG dictionary

In [2]:
bigg_dict = {}
with open("bigg_dict_21_09.txt") as f:
    for line in f:
        (key, val) = line.split(':[')
        val = val.replace(']', '')
        val = val.replace("'", '')
        val = val.replace('\n', '')
        val = [val]
        #print(key, val)
        bigg_dict[key] = val
        
print(len(bigg_dict))

624


In [8]:
#sec_bigg_dict = {}
with open('bigg_dict_23_09.txt') as f:
    for line in f:
        (key, val) = line.split('\t')
        key = key.replace(':', '')
        val = val.replace('[', '')
        val = val.replace(']', '')
        val = val.replace("'", '')
        val = val.replace('\n', '')
        val = [val]
        #print(key, val)
        bigg_dict[key] = val
print(len(bigg_dict))

958


In [9]:
with open('bigg_dict_total.txt', 'w') as f:
                for key, value in bigg_dict.items():
                    f.write('%s:\t%s\n' % (key, value))

### KEGG dictionary

In [10]:
kegg_dict = {}
with open("kegg_dict_21_09.txt") as f:
    for line in f:
        (key, val) = line.split(':[')
        val = val.replace(']', '')
        val = val.replace("'", '')
        val = val.replace('\n', '')
        val = [val]
        #print(key, val)
        kegg_dict[key] = val
        
print(len(kegg_dict))

624


In [12]:
#sec_bigg_dict = {}
with open('kegg_dict_23_09.txt') as f:
    for line in f:
        (key, val) = line.split('\t')
        key = key.replace(':', '')
        val = val.replace('[', '')
        val = val.replace(']', '')
        val = val.replace("'", '')
        val = val.replace('\n', '')
        val = [val]
        #print(key, val)
        kegg_dict[key] = val
print(len(kegg_dict))

958


In [13]:
with open('kegg_dict_total.txt', 'w') as f:
                for key, value in kegg_dict.items():
                    f.write('%s:\t%s\n' % (key, value))